In [85]:
import os
import numpy as np
import pandas as pd
from datetime import date

from sklearn.model_selection import KFold, train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, roc_auc_score, auc, roc_curve
from sklearn.preprocessing import MinMaxScaler

DATA_ROOT = "data/"

In [86]:
dfoff = pd.read_csv(os.path.join(DATA_ROOT,'train_offline.csv'))
dftest = pd.read_csv(os.path.join(DATA_ROOT,'test_offline.csv'))
dfoff = dfoff[~dfoff.Coupon_id.isna()]
dftest = dftest[~dftest.Coupon_id.isna()]
dfoff.reset_index(drop=True, inplace=True)
dftest.reset_index(drop=True, inplace=True)
print(dfoff.shape)
print(dftest.shape)

(746969, 7)
(306313, 6)


In [87]:
print(dfoff.head(10))
print(dftest.head(10))

   User_id  Merchant_id  Coupon_id Discount_rate  Distance  Date_received  \
0  1439408         2632     8591.0          20:1       0.0     20160217.0   
1  1439408         2632     1078.0          20:1       0.0     20160319.0   
2  1832624         3381     7610.0        200:20       0.0     20160429.0   
3  2029232         3381    11951.0        200:20       1.0     20160129.0   
4  2223968         3381     9776.0          10:5       2.0     20160129.0   
5    73611         2099    12034.0        100:10       NaN     20160207.0   
6   163606         1569     5054.0        200:30      10.0     20160421.0   
7  3273056         4833     7802.0        200:20      10.0     20160130.0   
8    94107         3381     7610.0        200:20       2.0     20160412.0   
9   253750         8390     7531.0          20:5       0.0     20160327.0   

   Date  
0   NaN  
1   NaN  
2   NaN  
3   NaN  
4   NaN  
5   NaN  
6   NaN  
7   NaN  
8   NaN  
9   NaN  
   User_id  Merchant_id  Coupon_id Discoun

In [88]:
def label_check(x):
    if np.isnan(x['Date_received']):
        return -1
    if not np.isnan(x["Date"]):
        td = pd.to_datetime(x['Date'], format='%Y%m%d') - pd.to_datetime(x['Date_received'], format='%Y%m%d')
        if td <= pd.Timedelta(15, 'D'):
            return 1
    return 0
dfoff["label"] = dfoff.apply(label_check, axis=1)
print(dfoff["label"].value_counts())
print(dfoff.head(20))

0    710665
1     36304
Name: label, dtype: int64
    User_id  Merchant_id  Coupon_id Discount_rate  Distance  Date_received  \
0   1439408         2632     8591.0          20:1       0.0     20160217.0   
1   1439408         2632     1078.0          20:1       0.0     20160319.0   
2   1832624         3381     7610.0        200:20       0.0     20160429.0   
3   2029232         3381    11951.0        200:20       1.0     20160129.0   
4   2223968         3381     9776.0          10:5       2.0     20160129.0   
5     73611         2099    12034.0        100:10       NaN     20160207.0   
6    163606         1569     5054.0        200:30      10.0     20160421.0   
7   3273056         4833     7802.0        200:20      10.0     20160130.0   
8     94107         3381     7610.0        200:20       2.0     20160412.0   
9    253750         8390     7531.0          20:5       0.0     20160327.0   
10   376492         1041    13490.0          30:5       2.0     20160127.0   
11  1964720   

In [89]:
def getWeekday(x):
    if (np.isnan(x)) or (x==-1):
        return x
    else:
        return pd.to_datetime(x, format = "%Y%m%d").dayofweek + 1

dfoff['weekday'] = dfoff['Date_received'].apply(getWeekday)
dftest['weekday'] = dftest['Date_received'].apply(getWeekday)

dfoff["weekend"] = dfoff["weekday"].apply(lambda x:1 if x in [6,7] else 0)
dftest["weekend"] = dfoff["weekday"].apply(lambda x:1 if x in [6,7] else 0)

In [90]:
weekdaycols = ['weekday_' + str(i) for i in range(1,8)]
print(weekdaycols)

tmpdf = pd.get_dummies(dfoff['weekday'].replace(-1, np.nan))
tmpdf.columns = weekdaycols
dfoff[weekdaycols] = tmpdf

tmpdf = pd.get_dummies(dftest['weekday'].replace(-1, np.nan))
tmpdf.columns = weekdaycols
dftest[weekdaycols] = tmpdf

['weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7']


In [91]:
#print(dfoff["Discount_rate"].head(10))
#print(dfoff["Discount_rate"].value_counts)
def getDiscountType(x):
    if x == 'null':
        return 'null'
    elif ':' in x:
        return 1
    else:
        return 0

def Convert_to_rate(x):
    if x == "null":
        return 1
    elif ":" in x:
        temp = x.split(":")
        return 1- float(temp[1])/float(temp[0])
    else:
        return float(x)
    
def getDiscountMan(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[0])
    else:
        return 0

def getDiscountJian(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[1])
    else:
        return 0

#每300最高省下金額
def getSavemoney(x):
    if ":" in x:
        return int(x.split(":")[1])
    elif x == "null":
        return 0
    else:
        return float(x) * 300

def processData(df):
    df['discount_rate'] = df['Discount_rate'].apply(Convert_to_rate)
    df['discount_man'] = df['Discount_rate'].apply(getDiscountMan)
    df['discount_jian'] = df['Discount_rate'].apply(getDiscountJian)
    df['discount_save'] = df['Discount_rate'].apply(getSavemoney)
    df['discount_type'] = df['Discount_rate'].apply(getDiscountType)
    df.loc[df.Distance.isna(), "Distance"] = 99
    return(df)

dfoff = processData(dfoff)
dftest = processData(dftest)
print(dfoff.head(20))

    User_id  Merchant_id  Coupon_id Discount_rate  Distance  Date_received  \
0   1439408         2632     8591.0          20:1       0.0     20160217.0   
1   1439408         2632     1078.0          20:1       0.0     20160319.0   
2   1832624         3381     7610.0        200:20       0.0     20160429.0   
3   2029232         3381    11951.0        200:20       1.0     20160129.0   
4   2223968         3381     9776.0          10:5       2.0     20160129.0   
5     73611         2099    12034.0        100:10      99.0     20160207.0   
6    163606         1569     5054.0        200:30      10.0     20160421.0   
7   3273056         4833     7802.0        200:20      10.0     20160130.0   
8     94107         3381     7610.0        200:20       2.0     20160412.0   
9    253750         8390     7531.0          20:5       0.0     20160327.0   
10   376492         1041    13490.0          30:5       2.0     20160127.0   
11  1964720         7884     6704.0          20:1      10.0     

In [92]:
mean_mer = dfoff.groupby(["Merchant_id"])["label"].mean().reset_index()
mean_mer.columns = ["Merchant_id", "Merchant_rate"]
dfoff = pd.merge(dfoff, mean_mer, on="Merchant_id", how="left")
dftest = pd.merge(dftest, mean_mer, on="Merchant_id", how="left")
print(dfoff.head(10))

   User_id  Merchant_id  Coupon_id Discount_rate  Distance  Date_received  \
0  1439408         2632     8591.0          20:1       0.0     20160217.0   
1  1439408         2632     1078.0          20:1       0.0     20160319.0   
2  1832624         3381     7610.0        200:20       0.0     20160429.0   
3  2029232         3381    11951.0        200:20       1.0     20160129.0   
4  2223968         3381     9776.0          10:5       2.0     20160129.0   
5    73611         2099    12034.0        100:10      99.0     20160207.0   
6   163606         1569     5054.0        200:30      10.0     20160421.0   
7  3273056         4833     7802.0        200:20      10.0     20160130.0   
8    94107         3381     7610.0        200:20       2.0     20160412.0   
9   253750         8390     7531.0          20:5       0.0     20160327.0   

   Date  label  weekday  weekend  ...  weekday_4  weekday_5  weekday_6  \
0   NaN      0        3        0  ...          0          0          0   
1   

In [93]:
import math
def Merchant_rate_replace0(df):
    temp = df["Merchant_rate"].median()
    #df.loc[np.isnan(df["Merchant_rate"]), "Merchant_rate"] = temp
    df["Merchant_rate"] = df["Merchant_rate"].apply(lambda x:temp if math.isnan(x) else x)
    return df
dftest = Merchant_rate_replace0(dftest)
print(dftest.head(5))

   User_id  Merchant_id  Coupon_id Discount_rate  Distance  Date_received  \
0  1439408         4663    11002.0        150:20       1.0     20160528.0   
1  1439408         2632     8591.0          20:1       0.0     20160613.0   
2  1439408         2632     8591.0          20:1       0.0     20160516.0   
3  2029232          450     1532.0          30:5       0.0     20160530.0   
4  2029232         6459    12737.0          20:1       0.0     20160519.0   

   weekday  weekend  weekday_1  weekday_2  ...  weekday_4  weekday_5  \
0        6        0          0          0  ...          0          0   
1        1        1          1          0  ...          0          0   
2        1        0          1          0  ...          0          0   
3        1        0          1          0  ...          0          0   
4        4        0          0          0  ...          1          0   

   weekday_6  weekday_7  discount_rate  discount_man  discount_jian  \
0          1          0       0.8

In [94]:
mean_Cou = dfoff.groupby(["Coupon_id"])["label"].mean().reset_index()
mean_Cou.columns = ["Coupon_id", "Coupon_rate"]
dfoff = pd.merge(dfoff, mean_Cou, on="Coupon_id", how="left")
dftest = pd.merge(dftest, mean_Cou, on="Coupon_id", how="left")
print(dfoff.head(10))

   User_id  Merchant_id  Coupon_id Discount_rate  Distance  Date_received  \
0  1439408         2632     8591.0          20:1       0.0     20160217.0   
1  1439408         2632     1078.0          20:1       0.0     20160319.0   
2  1832624         3381     7610.0        200:20       0.0     20160429.0   
3  2029232         3381    11951.0        200:20       1.0     20160129.0   
4  2223968         3381     9776.0          10:5       2.0     20160129.0   
5    73611         2099    12034.0        100:10      99.0     20160207.0   
6   163606         1569     5054.0        200:30      10.0     20160421.0   
7  3273056         4833     7802.0        200:20      10.0     20160130.0   
8    94107         3381     7610.0        200:20       2.0     20160412.0   
9   253750         8390     7531.0          20:5       0.0     20160327.0   

   Date  label  weekday  weekend  ...  weekday_5  weekday_6  weekday_7  \
0   NaN      0        3        0  ...          0          0          0   
1   

In [95]:
import math
def Coupon_rate_replace0(df):
    temp = df["Coupon_rate"].median()
    #df.loc[np.isnan(df["Merchant_rate"]), "Merchant_rate"] = temp
    df["Coupon_rate"] = df["Coupon_rate"].apply(lambda x:temp if math.isnan(x) else x)
    return df
dftest = Coupon_rate_replace0(dftest)
print(dftest.head(5))

   User_id  Merchant_id  Coupon_id Discount_rate  Distance  Date_received  \
0  1439408         4663    11002.0        150:20       1.0     20160528.0   
1  1439408         2632     8591.0          20:1       0.0     20160613.0   
2  1439408         2632     8591.0          20:1       0.0     20160516.0   
3  2029232          450     1532.0          30:5       0.0     20160530.0   
4  2029232         6459    12737.0          20:1       0.0     20160519.0   

   weekday  weekend  weekday_1  weekday_2  ...  weekday_5  weekday_6  \
0        6        0          0          0  ...          0          1   
1        1        1          1          0  ...          0          0   
2        1        0          1          0  ...          0          0   
3        1        0          1          0  ...          0          0   
4        4        0          0          0  ...          0          0   

   weekday_7  discount_rate  discount_man  discount_jian  discount_save  \
0          0       0.866667  

In [96]:
df = dfoff[dfoff['label'] != -1].copy()
print(df.shape, dfoff.shape)

(746969, 24) (746969, 24)


In [108]:
def split_train_valid(x, cut="20160423"):
    if pd.to_datetime(x, format="%Y%m%d") < pd.to_datetime(cut, format="%Y%m%d"):
        return True
    else:
        return False
df["IsTrain"] = dfoff["Date_received"].apply(split_train_valid)
df_train = df[df["IsTrain"]]
df_valid = df[~df["IsTrain"]]
df_train.reset_index(drop=True, inplace=True)
df_valid.reset_index(drop=True, inplace=True)

In [180]:
print("Train size: {}, #positive: {}".format(len(df_train), df_train["label"].sum()))
print("Valid size: {}, #positive: {}".format(len(df_valid), df_valid["label"].sum()))
feature = df.columns
#print(feature)
Choose_feature = ['Distance',
                  'weekend', 
                  'discount_rate',
                  'discount_man', 
                  'discount_save',
                  'discount_type',
                  'discount_jian',
                  "Merchant_rate",
                  "weekday_7"]
print(len(Choose_feature), Choose_feature)

Train size: 703894, #positive: 33963
Valid size: 43075, #positive: 2341
8 ['Distance', 'weekend', 'discount_rate', 'discount_man', 'discount_save', 'discount_jian', 'Merchant_rate', 'weekday_7']


In [181]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_curve
predictors = Choose_feature
rf = RandomForestClassifier(n_estimators=20, min_samples_split=10, min_samples_leaf=5, 
                            max_features=4, max_depth=3, bootstrap=True)
onehot = OneHotEncoder()
lr = LogisticRegression(solver='lbfgs', max_iter=1000)
train_X = df_train[predictors]
train_Y = df_train["label"]
val_X = df_valid[predictors]
val_Y = df_valid["label"]
rf.fit(train_X, train_Y)
onehot.fit(rf.apply(train_X))
lr.fit(onehot.transform(rf.apply(val_X)), val_Y)

C:\Users\Hsin-PC\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [182]:
#篩選重要特徵
feats = pd.Series(data=rf.feature_importances_, index=predictors)
feats = feats.sort_values(ascending=False)
feats

Merchant_rate    0.740586
Distance         0.099659
discount_man     0.084219
discount_rate    0.048021
discount_jian    0.016168
weekday_7        0.005879
discount_save    0.005468
weekend          0.000000
dtype: float64

In [183]:
y_valid_pred = lr.predict_proba(onehot.transform(rf.apply(val_X)))
valid1 = df_valid.copy()
valid1['pred_prob'] = y_valid_pred[:, 1]

In [184]:
from sklearn.metrics import roc_auc_score, accuracy_score
auc_score = roc_auc_score(y_true=df_valid.label, y_score=y_valid_pred[:,1])
acc = accuracy_score(y_true=df_valid.label, y_pred=y_valid_pred.argmax(axis=1))
print("Validation AUC: {:.3f}, Accuracy: {:.3f}".format(auc_score, acc))

Validation AUC: 0.902, Accuracy: 0.949


In [185]:
targetset = dftest.copy()
print(targetset.shape)
testset = targetset[predictors].copy()

test_X = dftest[predictors]
y_test_pred = lr.predict_proba(onehot.transform(rf.apply(test_X)))
test1 = testset.copy()
test1['pred_prob'] = y_test_pred[:, 1]
print(test1.shape)
print(test1.head(10))
print(y_test_pred[:, 0])
print(y_test_pred[:, 1])

(306313, 22)
(306313, 9)
   Distance  weekend  discount_rate  discount_man  discount_save  \
0       1.0        0       0.866667           150           20.0   
1       0.0        1       0.950000            20            1.0   
2       0.0        0       0.950000            20            1.0   
3       0.0        0       0.833333            30            5.0   
4       0.0        0       0.950000            20            1.0   
5      99.0        1       0.800000            50           10.0   
6       1.0        0       0.950000            20            1.0   
7       0.0        1       0.833333            30            5.0   
8      99.0        0       0.866667           150           20.0   
9       0.0        1       0.750000            20            5.0   

   discount_jian  Merchant_rate  weekday_7  pred_prob  
0             20       0.009023          0   0.003036  
1              1       0.031250          0   0.021593  
2              1       0.031250          0   0.021593  
3 

In [186]:
output = pd.concat((targetset[["User_id", "Coupon_id", "Date_received"]], test1["pred_prob"]), axis=1)
print(output.shape)

output.loc[:, "User_id"] = output["User_id"].apply(lambda x:str(int(x)))
output.loc[:, "Coupon_id"] = output["Coupon_id"].apply(lambda x:str(int(x)))
output.loc[:, "Date_received"] = output["Date_received"].apply(lambda x:str(int(x)))
output["uid"] = output[["User_id", "Coupon_id", "Date_received"]].apply(lambda x: '_'.join(x.values), axis=1)
output.reset_index(drop=True, inplace=True)

(306313, 4)


In [187]:
out = output.groupby("uid", as_index=False).mean()
out = out[["uid", "pred_prob"]]
out.columns = ["uid", "label"]
out.head()

,uid,label
0,1000020_2705_20160519,0.159990
1,1000020_8192_20160513,0.018847
2,1000065_1455_20160527,0.273250
3,1000085_8067_20160513,0.010967
4,1000086_2418_20160613,0.066748


In [188]:
out.to_csv("Result/fit_result_12.csv", sep=',', encoding='utf-8', index=False)